In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook

import asdf
from astropy.io import fits
from astropy.wcs import WCS
import ndcube

In [ ]:
test = fits.open("test_data/ib6o23rsq_flt.fits")

In [ ]:
#im = plt.imshow(test[1].data[250:350,100:600])
im = plt.imshow(test[1].data)
im.set_clim(0, 100)

In [ ]:
# Finding where the object is in the direct image

direct = fits.open("test_data/F140W/ib6o23rtq_flt.fits")

im2 = plt.imshow(direct[1].data[270:320,140:200])
im2.set_clim(0, 100)

In [ ]:
# Use the direct image bounding box to extract the 2D spectrum

from extraction import extract_2d_spectrum

sliced_data = extract_2d_spectrum(test[1].data, 160, 285, 172, 300)

In [ ]:
sliced_data.shape

In [ ]:
plt.imshow(sliced_data)